In [1]:
from src.json_models.src.model_generator import ModelGenerator

In [2]:
gen = ModelGenerator(json_path="/home/andrewheschl/PycharmProjects/ClassificationPipeline/models/mobilenets/mobilenet.json")

In [3]:
model = gen.get_model()

In [4]:
import torch
data = torch.ones((1, 3, 320, 320))
out = model(data)

file = f"/home/andrewheschl/Documents/Dataset/ClassPipeline/model_topology.onnx"
torch.onnx.export(model, data, file, verbose=False)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================


/home/andrewheschl/anaconda3/envs/class/lib/python3.8/site-packages/torch/onnx/utils.py:1636: UserWarning: The exported ONNX model failed ONNX shape inference.The model will not be executable by the ONNX Runtime.If this is unintended and you believe there is a bug,please report an issue at https://github.com/pytorch/pytorch/issues.Error reported by strict ONNX shape inference: [ShapeInferenceError] Shape inference error(s): (op_type:MaxPool, node name: /self_modules.0/stem_conv/branches.0/ch_maxpool/MaxPool): [ShapeInferenceError] Attribute strides has incorrect size
(op_type:Add, node name: /self_modules.0/stem_conv/branches.0/Add): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:MaxPool, node name: /self_modules.0/stem_conv/branches.1/ch_maxpool/MaxPool): [ShapeInferenceError] Attribute strides has incorrect size
(op_type:Add, node name: /self_modules.0/stem_conv/branches.1/Add): [TypeInferenceError] Input 0 expected to have type but instead is null
(o

In [5]:
out.shape

torch.Size([1, 7])